In [33]:
import pandas as pd
import numpy as np

# 读入数据（任选一种方式：本地 or 在线）
path = r"D:\Pokemon.csv"                 # 如果你已下载到本地，改成你的路径

df = pd.read_csv(path, encoding="latin1", sep=None, engine="python")


# 规范列名（不同来源可能大小写/空格不同，这里做个兼容）
def pick(df, candidates):
    for c in candidates:
        if c in df.columns: return c
    raise KeyError(f"列不存在：{candidates}")

COL_T1  = pick(df, ["Type 1","type1","Type1"])
COL_T2  = pick(df, ["Type 2","type2","Type2"])
COL_ATK = pick(df, ["Attack","attack","ATK"])
COL_GEN = pick(df, ["Generation","generation","Gen"])
COL_LEG = pick(df, ["Legendary","legendary","isLegendary"])

df.shape, df.head()


((808, 13),
      #                   Name Type 1  Type 2  Total    HP  Attack  Defense  \
 0  1.0              Bulbasaur  Grass  Poison  318.0  45.0    49.0     49.0   
 1  2.0                Ivysaur  Grass  Poison  405.0  60.0    62.0     63.0   
 2  3.0               Venusaur  Grass  Poison  525.0  80.0    82.0     83.0   
 3  3.0  VenusaurMega Venusaur  Grass  Poison  625.0  80.0   100.0    123.0   
 4  4.0             Charmander   Fire     NaN  309.0  39.0    52.0     43.0   
 
    Sp. Atk  Sp. Def  Speed Generation Legendary  
 0     65.0     65.0   45.0          1     FALSE  
 1     80.0     80.0   60.0          1     FALSE  
 2    100.0    100.0   80.0          1     FALSE  
 3    122.0    120.0   80.0          1     FALSE  
 4     60.0     50.0   65.0          1     FALSE  )

In [34]:
before = len(df)
if before >= 2:
    df = df.iloc[:-2].copy()   # 只保留到倒数第3行
else:
    df = df.iloc[0:0].copy()   # 不足2行时结果设为空表
print(f"Rows: {before} -> {len(df)}")


Rows: 808 -> 806


In [35]:
import pandas as pd

# 原列
t2_raw = df[COL_T2]

# 统一为字符串便于统计；缺失标成 [Missing]，纯空白标成 [Blank]
raw_labels = (
    t2_raw.astype(object)  # 保留 None/NaN
          .where(~t2_raw.isna(), '[Missing]')
          .astype(str).str.strip()
          .replace({'': '[Blank]'})
)

print("【原样口径：全部取值与频数】")
vc_raw = raw_labels.value_counts(dropna=False)
print(vc_raw)

# 仅看“非常见/疑似异常”的：排除合法全集与 [Missing]/[Blank]，其余全部列出
valid_types = {
    'Bug','Dark','Dragon','Electric','Fairy','Fighting','Fire','Flying','Ghost',
    'Grass','Ground','Ice','Normal','Poison','Psychic','Rock','Steel','Water'
}
mask_suspect_raw = ~raw_labels.isin(valid_types | {'[Missing]','[Blank]'})
suspect_counts_raw = raw_labels[mask_suspect_raw].value_counts()

print("\n【原样口径：非常见取值（包含只出现 1 次者）】")
print(suspect_counts_raw if not suspect_counts_raw.empty else "（无）")

# 如果你想只看“只出现 1 次”的值（包括合法与非法）
singletons = raw_labels.value_counts()
singletons = singletons[singletons == 1]
print("\n【原样口径：出现仅 1 次的取值】")
print(singletons if not singletons.empty else "（无）")


【原样口径：全部取值与频数】
Type 2
[Missing]    384
Flying        98
Poison        37
Ground        35
Psychic       33
Fighting      26
Grass         25
Fairy         23
Steel         22
Dark          20
Dragon        18
Ghost         14
Water         14
Rock          14
Ice           14
Fire          12
Electric       6
Normal         4
Bug            3
273            1
0              1
A              1
BBB            1
Name: count, dtype: int64

【原样口径：非常见取值（包含只出现 1 次者）】
Type 2
0      1
273    1
A      1
BBB    1
Name: count, dtype: int64

【原样口径：出现仅 1 次的取值】
Type 2
273    1
0      1
A      1
BBB    1
Name: count, dtype: int64


In [36]:
import numpy as np
import unicodedata

# 列名
# COL_T2 = 'Type 2'

def norm_casefold(x: object) -> str | None:
    """只用于匹配：全角->半角，去首尾空白，大小写无关（casefold）。"""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = unicodedata.normalize('NFKC', str(x)).strip()
    return s.casefold() if s != "" else ""   # 空白保留为 ""，后面不会当异常

# 需要去除（置 NaN）的取值集合（统一到 casefold 口径）
ban_set = { '0', '273', 'a', 'bbb' }

# 构造掩码：非缺失、非空白，且规范化后命中 ban_set
t2_norm = df[COL_T2].map(norm_casefold)
mask_ban = t2_norm.notna() & (t2_norm != "") & t2_norm.isin(ban_set)

print("命中需要去除的条数：", int(mask_ban.sum()))
print("命中的原值示例：", df.loc[mask_ban, COL_T2].head(10).tolist())

# 去除：置为 NaN（如果你想直接删整行，见下面“可选”）
df.loc[mask_ban, COL_T2] = np.nan


命中需要去除的条数： 4
命中的原值示例： ['0', '273', 'A', 'BBB']


In [37]:
# 统计重复行数
dup_count = df.duplicated().sum()

# 列出所有重复的行（包含第一次出现的）
dups_all = df[df.duplicated(keep=False)]

# 如果只想看“重复部分的值”，可以用 drop_duplicates() 去重
dups_unique = dups_all.drop_duplicates()

print(f"去重条数：{dup_count}；")
print("\n>>> 重复的行（包含所有重复出现）:")
print(dups_all)

print("\n>>> 重复的行（每组只保留一次）:")
print(dups_unique)

# 去重后再赋值
df = df.drop_duplicates()


去重条数：5；

>>> 重复的行（包含所有重复出现）:
         #       Name  Type 1  Type 2  Total    HP  Attack  Defense  Sp. Atk  \
14    11.0    Metapod     Bug     NaN  205.0  50.0    20.0     55.0     25.0   
15    11.0    Metapod     Bug     NaN  205.0  50.0    20.0     55.0     25.0   
21    17.0  Pidgeotto  Normal  Flying  349.0  63.0    60.0     55.0     50.0   
23    17.0  Pidgeotto  Normal  Flying  349.0  63.0    60.0     55.0     50.0   
184  168.0    Ariados     Bug  Poison  390.0  70.0    90.0     70.0     60.0   
185  168.0    Ariados     Bug  Poison  390.0  70.0    90.0     70.0     60.0   
186  168.0    Ariados     Bug  Poison  390.0  70.0    90.0     70.0     60.0   
187  168.0    Ariados     Bug  Poison  390.0  70.0    90.0     70.0     60.0   

     Sp. Def  Speed Generation Legendary  
14      25.0   30.0          1     FALSE  
15      25.0   30.0          1     FALSE  
21      50.0   71.0          1     FALSE  
23      50.0   71.0          1     FALSE  
184     60.0   40.0          2     

In [38]:
# —— 将 Attack 列转成数值并做“高端温莎化”截断 ——
import numpy as np

# 1) 字符清洗 → 数值
s_raw = df[COL_ATK].astype(str).str.strip()
s_clean = (
    s_raw.str.replace(',', '', regex=False)        # 去掉千分位逗号，如 "1,234"
         .replace({'': np.nan, 'NA': np.nan, 'N/A': np.nan,
                   '-': np.nan, '--': np.nan, '—': np.nan})
)
atk_num = pd.to_numeric(s_clean, errors='coerce')  # 无法解析的变为 NaN

# 可选：看看有多少无法解析
coerced_cnt = ((atk_num.isna()) & (s_clean.notna())).sum()    
print(f"[INFO] Attack 可用数值：{atk_num.notna().sum()}，无法解析：{coerced_cnt}")

# 回写为数值列
df[COL_ATK] = atk_num

# 2) 若没有数值就直接跳过
ser = df[COL_ATK].dropna()
if ser.empty:
    print("[WARN] Attack 列没有可用数值，跳过 IQR 处理")
else:
    # 3) IQR 上端阈值（只截高端，符合“过高异常值”的要求）
    q1, q3 = ser.quantile([0.25, 0.75]).values
    iqr = q3 - q1
    upper = q3 + 1.5 * iqr

    mask_hi = df[COL_ATK] > upper
    n_hi = int(mask_hi.sum())
    df.loc[mask_hi, COL_ATK] = upper

    print(f"[CLEAN] Attack 高端截断 {n_hi} 行；阈值上限 = {upper:.2f}")
    display(df[COL_ATK].describe())


[INFO] Attack 可用数值：800，无法解析：1
[CLEAN] Attack 高端截断 9 行；阈值上限 = 167.50


count    800.000000
mean      79.110625
std       32.445670
min        5.000000
25%       55.000000
50%       75.000000
75%      100.000000
max      167.500000
Name: Attack, dtype: float64

In [39]:
gen_boollike = df[COL_GEN].astype(str).str.strip().str.lower().isin(["true","false"])
leg_numeric  = pd.to_numeric(df[COL_LEG], errors="coerce").notna()
swap_idx = gen_boollike & leg_numeric
print("检测到置换行数：", int(swap_idx.sum()))

if swap_idx.any():
    df.loc[swap_idx, [COL_GEN, COL_LEG]] = df.loc[swap_idx, [COL_LEG, COL_GEN]].values

df[COL_GEN] = pd.to_numeric(df[COL_GEN], errors="coerce").astype("Int64")
df[COL_LEG] = df[COL_LEG].astype(str).str.strip().str.lower().map({"true": True, "false": False})
df[[COL_GEN, COL_LEG]].head(10)


检测到置换行数： 2


,Generation,Legendary
0,1,False
1,1,False
2,1,False
3,1,False
4,1,False
5,1,False
6,1,False
7,1,False
8,1,False
9,1,False


In [40]:
out_path = "Pokemon_clean_utf8.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("已保存：", out_path)


已保存： Pokemon_clean_utf8.csv
